In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn
import quimb as qu
import cotengra as ctg
import autoray as ar
import register_ as reg
import algo_cooling as algo
import quf
import time
import numpy as np
from quimb.tensor.belief_propagation.l2bp import L2BP
from tqdm import tqdm


import matplotlib as mpl
import matplotlib.pyplot as plt



In [2]:
reg.reg_complex_svd()
import torch
to_backend = algo.backend_torch(device = "cpu", dtype = torch.float64, requires_grad=False)
to_backend_c = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)

opt = algo.opt_(progbar=False, max_time="rate:1e9", max_repeats=128, optlib="cmaes")


In [3]:
info_bp = {}

In [4]:
#ITF params
J = 1
h = 3.05
dt = 0.05
info_bp |= {"h":h, "J":J, "dt":dt }


In [5]:
Lx, Ly = 3, 4
L = Lx * Ly
edges = qtn.edges_2d_square(Lx=Lx, Ly=Ly, cyclic=False)

sites = sorted({ (site,) for edge in edges for site in edge})
N = len(sites)
info_bp |= {"Lx":Lx, "Ly":Ly, "L":L, "edges":edges, "sites":sites}


In [6]:
map_1d_2d = { Lx*j+i: (i,j)   for i in range(Lx) for j in range(Ly)}
map_2d_1d = { (i,j):Lx*j+i   for i in range(Lx) for j in range(Ly)}

inds_k = [ f"k{i}"  for i in range(L)]
inds_b = [ f"b{i}"  for i in range(L)]
ind_k_2d = []
ind_b_2d = []
for i in range(L):
        x,y = map_1d_2d[i]
        ind_k_2d.append(f"k{x},{y}")
        ind_b_2d.append(f"b{x},{y}")
    
info_bp |= {"inds_k":inds_k, "inds_b":inds_b, "ind_k_2d":ind_k_2d, "ind_b_2d":ind_b_2d}


In [7]:
dt = 0.1

In [8]:
from scipy.linalg import expm
mpo = quf.mpo_ITF_2d(Lx, Ly, data_type="float64", chi=200, cutoff_val=1.0e-12, field=h, sign="-", print_=False)
mpo
Ham_ = np.array(mpo.to_dense(inds_k, inds_b), dtype="complex128")

U_matrix = expm(-1j * Ham_ * dt)
U_matrix = to_backend_c(U_matrix)
print("U shape:", U_matrix.shape, dt)
print("Unitary?", np.allclose(U_matrix.conj().T @ U_matrix, np.eye(Ham_.shape[0])))


pepo_st = quf.pepo_trotter_ITF(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=dt)
pepo_st
pepo_ft_ = quf.pepo_trotter_ITF_fourth(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=dt)
pepo_ft_
pepo_sixth = quf.pepo_trotter_ITF_sixth(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=dt, bond_dim=20)
pepo_sixth
pepo_eigth = quf.pepo_trotter_ITF_eigth(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=dt, bond_dim=10,
                                       bond_dim_=4,)
pepo_eigth

pepo_st_m = pepo_st.to_dense(ind_b_2d, ind_k_2d)
pepo_ft_m = pepo_ft_.to_dense(ind_b_2d, ind_k_2d)
pepo_sixth_m = pepo_sixth.to_dense(ind_b_2d, ind_k_2d)
pepo_eigth_m = pepo_eigth.to_dense(ind_b_2d, ind_k_2d)

F = algo.gate_fidelity(pepo_st_m, U_matrix)
print("F-second order", F, 1- F)

F = algo.gate_fidelity(pepo_ft_m, U_matrix)
print("F-fourth order", F, 1- F)

F = algo.gate_fidelity(pepo_sixth_m, U_matrix)
print("F-sixth order", F, 1- F)
F = algo.gate_fidelity(pepo_eigth_m, U_matrix)
print("F-eigth order", F, 1- F)

model= -* \sum<ij> Z_iZ_j + -*3.05 X
U shape: torch.Size([4096, 4096]) 0.1
Unitary? True
F-second order 0.9994650280650087 0.0005349719349913018
F-fourth order 0.9999903880155091 9.611984490898173e-06
F-sixth order 0.9999999999810121 1.8987922345559127e-11
F-eigth order 0.9999999997841984 2.1580159881295913e-10


In [ ]:
from scipy.linalg import expm


mpo = quf.mpo_ITF_2d(Lx, Ly, data_type="float64", chi=200, cutoff_val=1.0e-12, field=h, sign="-", print_=False)
mpo
Ham_ = np.array(mpo.to_dense(inds_k, inds_b), dtype="complex128")

F_second = []
F_fourth = []
F_sixth = []

t_l = []
for t in [0.05, 0.06, 0.07, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    print(t)
    t_l.append( t )
    U_matrix = expm(-1j * Ham_ * t)
    U_matrix = to_backend_c(U_matrix)

    
    
    pepo_st = quf.pepo_trotter_ITF(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=t)
    pepo_ft_ = quf.pepo_trotter_ITF_fourth(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=t)
    pepo_sixth = quf.pepo_trotter_ITF_sixth(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, 
                                            J=J, delta=t, bond_dim=40)


    
    pepo_st_m = pepo_st.to_dense(ind_b_2d, ind_k_2d)
    pepo_ft_m = pepo_ft_.to_dense(ind_b_2d, ind_k_2d)
    pepo_sixth_m = pepo_sixth.to_dense(ind_b_2d, ind_k_2d)

    
    F = algo.gate_fidelity(pepo_st_m, U_matrix)
    F_second.append(  1-F )
    
    F = algo.gate_fidelity(pepo_ft_m, U_matrix)
    F_fourth.append(  1-F )
    
    F = algo.gate_fidelity(pepo_sixth_m, U_matrix)
    F_sixth.append(  1-F )
    


model= -* \sum<ij> Z_iZ_j + -*3.05 X
0.05
0.06
0.07


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 5))  # compact and clean figure



plt.plot(
    t_l, F_second, zorder=5, label=r"second",color="orange",
    markersize=12, linewidth=2.2, alpha=1., marker='*', linestyle='-',
    markeredgecolor="whitesmoke", mfc="orange", markeredgewidth=1.4
)

plt.plot(
    t_l, F_fourth, zorder=5, label=r"fourth",color="darkblue",
    markersize=12, linewidth=2.2, alpha=1., marker='*', linestyle='-',
    markeredgecolor="whitesmoke", mfc="magenta", markeredgewidth=1.4
)

plt.plot(
    t_l, F_sixth, zorder=5, label=r"sixth",color="gray",
    markersize=12, linewidth=2.2, alpha=1., marker='*', linestyle='-',
    markeredgecolor="whitesmoke", mfc="darkgreen", markeredgewidth=1.4
)


plt.yscale('log')

# Set axis limits
# plt.xlim(0, max(t_l))        # adjust x-axis range if needed
plt.ylim(1.e-6, 2.e+0)            # adjust y-axis range for log scale

plt.ylabel(r"$ 1- \mathrm{Tr}\!\left(U(t)^\dagger V_{order}\right) / 2^L$", fontsize=12)
plt.xlabel("time: t", fontsize=12)

plt.grid(True, linestyle='--', linewidth=0.3, alpha=0.5)
plt.tick_params(width=0.8, labelsize=12)
for spine in plt.gca().spines.values():
    spine.set_linewidth(1.)


plt.title(rf"Ising Model, $J=1, h=3.05$, $L = {3} \times {4}$,", fontsize=14, color='darkred', pad=15)  # pad adds space above the plot

plt.grid(color='gray', linestyle='-', linewidth=1., alpha=1.)
plt.legend(loc='best', frameon=True, shadow=True, fontsize=14)
plt.tight_layout()
plt.show()
